## Memristor Function

In [1]:
import numpy as np
import math as math

In [2]:
def memristor(V_total, di, AJ):
    NL, NR, NB, N = (1, 1, 10, 12)
    
    l1 = 1                      # Cociente de movilidad de celdas centrales respecto a las del lado derecho  ## Ratio
    l2 = 1                      # Cociente de movilidad de celdas del lado izquierdo respecto a las del lado derecho  ## Ratio
    fm = 16                     # Scale factor
  
    VBB, VBAL, VBAR = ((l1*fm), (l2*fm), (fm))    #Parametros - movilidad de vacancias en la zona central, izq, der
    
    AB = AJ[2] #k1*fr                    #Resistividad local zona central, izq, der
    AL = AJ[0] #k2*fr                    
    AR = AJ[1] #fr
    
    n = list(range(0,N))
    vb= list(range(0,N))
    A = list(range(0,N))
    rho=list(range(0,N))
    
    #Influencia de la temperatura
    
    T= 273
    #Conmutacion
    C1=0.00366300366300366
    
    #Conduccion
    S= 0  #(-0.17872) #0
    P= 0  #0.25    #0
    To= 0 #273    #0

    for i in n:
        if i < NL:
            vb[i]=VBAL
            A[i]=AL
        elif i >=len(n)-NR:
            vb[i]=VBAR
            A[i]=AR
        else:
            vb[i]=VBB
            A[i]=AB
   
    delta, piip1, pip1i, piim1, pim1i, deltap, deltam = (list(range(0,N)) for i in range(7))

    rho_total = 0
    for i in n:
        rho[i]=di[i]*A[i] #(T**S)*math.exp((To/T)**P)
        rho_total= rho_total + rho[i]
    i_total= V_total/rho_total

    #Calculo de las probabilidades de transicion entre sitios   im1 <-> i <-> ip1 
    for i in n:
        delta[i]=0
        
        #Condiciones de borde extremo derecho
        if (i==(N-1)):
                piip1[i]= 0
                pip1i[i]= 0
                piim1[i]= di[i]  *(1-di[i-1]) *math.exp(-vb[i]/(T*C1)  -i_total* rho[i]/2  ) 
                pim1i[i]= di[i-1]*(1-di[i])   *math.exp(-vb[i-1]/(T*C1) +i_total* rho[i-1]/2)

                deltap[i] = piip1[i]-pip1i[i]
                deltam[i] = piim1[i]-pim1i[i]

                if (deltap[i]>0):
                    deltap[i]= min(deltap[i],di[i])
                else:
                    deltap[i]=-min(-deltap[i],1-di[i])

                if (deltam[i]>0):
                    deltam[i]= min(deltam[i],di[i],1-di[i-1])
                else:
                    deltam[i]=-min(-deltam[i],di[i-1],1-di[i])

                delta[i]=deltap[i]+deltam[i]

        #Condiciones de borde extremo izquierdo
        elif (i==0):
                piip1[i]= di[i]  *(1-di[i+1]) *math.exp(-vb[i]/(T*C1)   +i_total* rho[i]/2  )
                pip1i[i]= di[i+1]*(1-di[i])   *math.exp(-vb[i+1]/(T*C1) -i_total* rho[i+1]/2)
                piim1[i]= 0
                pim1i[i]= 0

                deltap[i] = piip1[i]-pip1i[i]
                deltam[i] = piim1[i]-pim1i[i]

                if (deltap[i]>0):
                    deltap[i]= min(deltap[i],di[i],1-di[i+1])
                else:
                    deltap[i]=-min(-deltap[i],di[i+1],1-di[i])

                if (deltam[i]>0):
                    deltam[i]= min(deltam[i],di[i])
                else:
                    deltam[i]=-min(-deltam[i],1-di[i])

                delta[i]=deltap[i]+deltam[i]

        #Cálculo en el bulk
        else:
                piip1[i]= di[i]  *(1-di[i+1]) *math.exp(-vb[i]/(T*C1)   +i_total* rho[i]/2  )
                pip1i[i]= di[i+1]*(1-di[i])   *math.exp(-vb[i+1]/(T*C1) -i_total* rho[i+1]/2)
                piim1[i]= di[i]  *(1-di[i-1]) *math.exp(-vb[i]/(T*C1)   -i_total* rho[i]/2  )
                pim1i[i]= di[i-1]*(1-di[i])   *math.exp(-vb[i-1]/(T*C1) +i_total* rho[i-1]/2)

                deltap[i] = piip1[i]-pip1i[i]
                deltam[i] = piim1[i]-pim1i[i]

                if (deltap[i]>0):
                    deltap[i]= min(deltap[i],di[i],1-di[i+1])
                else:
                    deltap[i]=-min(-deltap[i],di[i+1],1-di[i])

                if (deltam[i]>0):
                    deltam[i]= min(deltam[i],di[i],1-di[i-1])
                else:
                    deltam[i]=-min(-deltam[i],di[i-1],1-di[i])

                delta[i]=deltap[i]+deltam[i]

    #Se actualiza la densidad local de vacancias di(i)
    for i in n:
        di[i]=di[i]-delta[i]
        if di[i]<0:
            di[i]=0
    
    return(di)